In [ ]:
import os
import pandas as pd
import math
import numpy as np
from tqdm import tqdm  
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
def unpack(parte):
    good_stuff = [y for x,y in parte]    
    return good_stuff

In [ ]:
def phasher(phash,ref_phashes):
    same = {}
    for index,ref_phash in enumerate(ref_phashes):
        similarity = same_same(phash,ref_phash)
        if similarity >= the_line:
            same[index] = similarity       
    same_list=[]    
    for key in same:
        value = same[key]
        same_list.append([value,key])
    same_list.sort(reverse=True)
    same_list = unpack(same_list)
    same_list = [int(i) for i in same_list]
    same_list = train_df.loc[same_list,].label_group.to_numpy()
    return same_list[0]

In [ ]:
def self_phasher(phashes):
    same={}
    pumpkina = len(phashes)
    for indexa in range(pumpkina):
        self_same = []
        pumpkin = len(phashes)-indexa
        pumpkinb = pumpkin-1
        phasha = phashes[indexa:]
        for index in range(pumpkin):
            similarity = same_same(phasha[0],phasha[pumpkinb-(pumpkinb-index)])
            self_same.append(similarity)
        same[indexa] = np.array(self_same)       
    return [item for sublist in same.values() for item in sublist]

In [ ]:
def  countSetBits(n):
    count = 0
    while (n):
        count += n & 1
        n >>= 1
    return count

In [ ]:
def same_same(phash1,phash2):
    result =phash1 ^ phash2
    result = countSetBits(int(result))
    result = 1-result/64
    return result

In [ ]:
def transformer(dictd,col1,col2):
    dict_list=[]
    for key in dictd:
        value = dictd[key]
        if not key.isnumeric() and len(key) > 1:
            dict_list.append([value,key])
    dict_list.sort(reverse=True)
    temp = pd.DataFrame(dict_list,columns=[col1,col2])
    return temp.set_index('words')

In [ ]:
def tf_idf(astring):
    t2 = Tokenizer()
    t2.fit_on_texts([astring])
    word_counts=transformer(t2.word_counts,'countd','words')
    total_words = len(word_counts)
    word_tf_idf={}
    for row in word_counts.iterrows():
        try:
            tf = list(row)[1][0]/total_words
            idf = word_docs.loc[row[0],].idf
            tf_idf_value = tf * idf
            word_tf_idf[row[0]]=tf_idf_value
        except:
            pass
    dict_list=[]
    for key in word_tf_idf:
        value = word_tf_idf[key]
        dict_list.append([value,key])
    dict_list.sort(reverse=True)
    return dict_list   

In [ ]:
def candy_man(batch_tokens,batch,y_hats):
    for indexa,title_tokens in enumerate(batch_tokens):
        award = {}
        for index,title_tokens2 in enumerate(ref_tf_idfs):
            local_award = 0
            for index2,(weight,word) in enumerate(title_tokens): 
                twerk = 0
                for index3,(weight2,word2) in enumerate(title_tokens2):
                    twerk+=1
                    if word == word2 and weight <= weight2:
                        local_award+=2 * weight
                        break
                    elif word == word2:
                        local_award+=1 * weight
                        break
                if twerk >= len(title_tokens2):
                    break
                
                
            if local_award > 0:
                award[index] = local_award
        award_list=[]
        label_list=[]
        for key in award:
            value = award[key]
            award_list.append([value,key])
        if len(award_list) > 0:
            award_list.sort(reverse=True)

        for (award,indexer) in award_list:
            label_list.append([award,train_df.loc[indexer,].label_group])
        y_hats[BATCH_SIZE*batch+indexa]= label_list

In [ ]:
def anotate_y_hat(yaya,refs):
    sampler,phash = yaya
    if len(sampler) == 0:
        clas = phasher(phash,refs)
    else:
        choco = unpack(sampler)
        getter = min(len(choco),1)
        clas = choco[:getter][0]
    return clas

In [ ]:
def annotate_postings(batch_test_class,batch,df,ref_classes_all,y_hat_postings):
    for indexa,test_class in enumerate(batch_test_class):
        local_postings=[]
        for index,ref_class in enumerate(ref_classes_all):              
                
            if test_class == ref_class:
                local_postings.append(df.loc[index,].posting_id)
                
        
        y_hat_postings[BATCH_SIZE*batch+indexa]= local_postings

In [ ]:
def prime_test():
    test_df['tf_idf']=test_df.title.map(lambda x: tf_idf(x))
    y_hats={}
    for batch in tqdm(range(TEST_BATCHES)):
        candy_man(test_df.tf_idf[batch * BATCH_SIZE : min(BATCH_SIZE*batch + BATCH_SIZE,len_data_test)],batch,y_hats)
    return y_hats

In [ ]:
def prime_train():
    y_hats={}
    for batch in tqdm(range(TRAIN_BATCHES)):
        candy_man(train_df.tf_idf[batch * BATCH_SIZE : min(BATCH_SIZE*batch + BATCH_SIZE,len_data_train)],batch,y_hats)
    return y_hats

In [ ]:
def predict_test(y_hats):
    test_df['y_hat_weights']=test_df.index.map(y_hats)
    test_df['yaya']=[test_df[['y_hat_weights']].values.tolist()[i] + test_df[['image_phash']].values.tolist()[i]
                     for i in range(0, len(test_df[['image_phash']].values.tolist()))]
    test_df['y_hat_labels']=test_df.yaya.map(lambda x: anotate_y_hat(x,train_df.image_phash))

    y_hat_postings={}
    ref_classes_all = test_df.y_hat_labels
    for batch in tqdm(range(TEST_BATCHES)):
        annotate_postings(test_df.y_hat_labels[batch * BATCH_SIZE : min(BATCH_SIZE*batch + BATCH_SIZE,len_data_test)],batch,test_df,ref_classes_all,y_hat_postings)
    return y_hat_postings

In [ ]:
def predict_train(y_hats):
    train_df['y_hat_weights']=train_df.index.map(y_hats)    
    train_df['yaya']=[train_df[['y_hat_weights']].values.tolist()[i] + train_df[['image_phash']].values.tolist()[i]
                     for i in range(0, len(train_df[['image_phash']].values.tolist()))]
    train_df['y_hat_labels']=train_df.yaya.map(lambda x: anotate_y_hat(x,train_df.image_phash))

    y_hat_postings={}
    ref_classes_all = train_df.y_hat_labels
    for batch in tqdm(range(TRAIN_BATCHES)):
        annotate_postings(train_df.y_hat_labels[batch * BATCH_SIZE : min(BATCH_SIZE*batch + BATCH_SIZE,len_data_train)],batch,train_df,ref_classes_all,y_hat_postings)
    return y_hat_postings

In [ ]:
def submit_test(y_hat_postings):
    test_df['y_hat_postings']=test_df.index.map(y_hat_postings)
    temp = test_df.groupby('image_phash').posting_id.agg('unique').to_dict()
    test_df['smoother'] = test_df.image_phash.map(temp)


    test_df['matches'] =[     ' '.join(np.unique([item for item in np.array(test_df[['smoother']].values[i][0]).flatten()] + [item for item in np.array(test_df[['y_hat_postings']].values[i][0],dtype=object).flatten()]))
                     for i in range(0, len(test_df[['y_hat_postings']].values.tolist()))      ]

                                          
    test_df[['posting_id','matches']].to_csv('submission.csv',index=False)

In [ ]:
def submit_train(y_hat_postings):
    train_df['y_hat_postings']=train_df.index.map(y_hat_postings)
    temp = train_df.groupby('image_phash').posting_id.agg('unique').to_dict()
    train_df['smoother'] = train_df.image_phash.map(temp)


    train_df['matches'] =[     ' '.join(np.unique([item for item in np.array(train_df[['smoother']].values[i][0]).flatten()] + [item for item in np.array(train_df[['y_hat_postings']].values[i][0],dtype=object).flatten()]))
                     for i in range(0, len(train_df[['y_hat_postings']].values.tolist()))      ]

                                          
   
    
    temp = train_df.groupby('label_group').posting_id.agg('unique').to_dict()
    train_df['target'] = train_df.label_group.map(temp)
    
    train_df['f1'] = train_df.apply(getMetric('matches'),axis=1)
    print('CV score for baseline =',train_df.f1.mean())
    train_df[['posting_id','matches']].to_csv('submission.csv',index=False)

In [ ]:
def scope_test():
    google_backend()
    predictions = test_df.y_hat_labels.to_numpy()
    for index,prediction in enumerate(predictions):
        matches = train_df.loc[train_df['label_group'] == prediction].index.values.tolist()
        plt.figure(figsize=(10,10))
        for i,match in enumerate(matches[:25]):
            image = train_df.iloc[match].image
            path = os.path.join(train_images,image)
            img = tf.keras.preprocessing.image.load_img(path) 
        
            plt.subplot(5,5,i+1)
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            plt.imshow(img, cmap=plt.cm.binary)
            plt.xlabel(prediction)
        plt.show()
        posting_id = test_df.iloc[index,:].posting_id
    
        image = test_df.iloc[index,:].image
        path = os.path.join(test_images,image)
        img = tf.keras.preprocessing.image.load_img(path)     
        plt.figure(figsize=(32,32))
        plt.subplot(5,5,1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(img, cmap=plt.cm.binary)
        plt.xlabel(posting_id)
        plt.show()

In [ ]:
def google_frontend():
    y_hats = prime_train()
    y_hat_postings=predict_train(y_hats)
    submit_train(y_hat_postings)
    scope_test()

In [ ]:
def google_backend():
    y_hats = prime_test()
    y_hat_postings=predict_test(y_hats)
    submit_test(y_hat_postings) 

In [ ]:
#https://www.kaggle.com/cdeotte
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,np.array(row[col].split())) )
        return 2*n / (len(row.target)+len(np.array(row[col].split())))
    return f1score

In [ ]:
train_df = pd.read_csv('../input/shopee-product-matching/train.csv')
test_df = pd.read_csv('../input/shopee-product-matching/test.csv')
train_images = '../input/shopee-product-matching/train_images'
test_images = '../input/shopee-product-matching/test_images'

In [ ]:
train_df['image_phash']= train_df.image_phash.apply(lambda x: '0x'+x)
test_df['image_phash']= test_df.image_phash.apply(lambda x: '0x'+x)
train_df['image_phash']=train_df.image_phash.apply(lambda x: int(x, 16))
test_df['image_phash']=test_df.image_phash.apply(lambda x: int(x, 16))
temp = train_df.groupby('label_group')['image_phash'].apply(list).to_dict()
train_df['phashes']= train_df.label_group.map(temp)
train_df['image_phash_weights']= train_df.phashes.apply(lambda x: self_phasher(x))
train_df['means']= train_df.image_phash_weights.apply(    lambda x:np.array(x).mean()     )
train_df['stds']= train_df.image_phash_weights.apply(     lambda x:np.array(x).std()     )
train_df['mins']= train_df.image_phash_weights.apply(     lambda x:np.array(x).min()     )
the_line = train_df.mins.mean()

In [ ]:
xs_train = train_df.title.to_numpy()
xs_test = test_df.title.to_numpy()
t = Tokenizer()
t.fit_on_texts(xs_train)

In [ ]:
total_docs = t.document_count
word_docs=transformer(t.word_docs,'docs','words')
word_docs['idf']=word_docs.docs.map(lambda x: math.log10(total_docs/    (x+0.00000000000000001)        ))

In [ ]:
len_data_train  = len(train_df)
len_data_test  = len(test_df)
BATCH_SIZE = 32
TRAIN_BATCHES = math.ceil(len_data_train/BATCH_SIZE)
TEST_BATCHES = math.ceil(len_data_test/BATCH_SIZE)

In [ ]:
train_df['tf_idf']=train_df.title.map(lambda x: tf_idf(x))
ref_tf_idfs=train_df.tf_idf

In [ ]:
ref_phashes = train_df.image_phash

In [ ]:
front = True
if len_data_test > 3:
    front = False

In [ ]:
if front:
    google_frontend()
else:
    google_backend()
